In [ ]:
__depends__ = ["../results/human_averages.csv", "../results/model.csv"]
__dest__ = ["../figures/model_v_human.pdf", "../results/model_v_human.tex"]

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy.optimize
import matplotlib.pyplot as plt
import seaborn as sns
import util

In [ ]:
# Load in the human data
human = pd.read_csv(__depends__[0]).set_index('stimulus').sort_index()
human.head()

In [ ]:
# Load in the model data
model = pd.read_csv(__depends__[1]).set_index('stimulus').sort_index()
model.head()

In [ ]:
# Fit the model to the human data using logistic regression
def f(beta):
    yp = util.sigmoid(model["y"], beta)
    err = np.sqrt(np.sum((yp - human["mean"]) ** 2))
    return err

res = scipy.optimize.minimize_scalar(f)
beta = res['x']
beta

In [ ]:
# Compute predictions for the fitted model
fitted_model = util.sigmoid(model["y"], beta)
fitted_model.head()

In [ ]:
# Compute the correlation between the raw model results and average human responses
corrs = util.bootcorr(model["y"], human["mean"])
corrs.head()

In [ ]:
# Compute the correlation between the fitted model results and the average human responses
fitted_corrs = util.bootcorr(fitted_model, human["mean"])
fitted_corrs.head()

In [ ]:
# Plot both the raw and fitted models against human data
sns.set_style("white")
sns.set_context("talk")

fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)

ax1.errorbar(
    model["y"], human["mean"], 
    yerr=[human["lerr"],  human["uerr"]], 
    marker='o', linestyle='')
ax1.set_xlabel("Raw model responses")
ax1.set_ylabel("Human responses")
ax1.set_xlim(0, 1)
ax1.set_ylim(0, 1)
corrstr = r"$r={corr:.2f},\ 95\%\ \mathrm{{CI}}\ [{lower:.2f}, {upper:.2f}]$".format(**corrs)
ax1.text(1, 0.02, corrstr, horizontalalignment='right')

ax2.errorbar(
    fitted_model, human["mean"], 
    yerr=[human["lerr"],  human["uerr"]], 
    marker='o', linestyle='')
ax2.set_xlabel("Fitted model responses")
ax2.set_xlim(0, 1)
corrstr = r"$r={corr:.2f},\ 95\%\ \mathrm{{CI}}\ [{lower:.2f}, {upper:.2f}]$".format(**fitted_corrs)
ax2.text(1, 0.02, corrstr, horizontalalignment='right')

sns.despine()
fig.set_size_inches(16, 6)
plt.tight_layout()
plt.savefig(__dest__[0])

In [ ]:
# Save out the correlations and fitted parameter values 
# as LaTeX commands for use in the paper
def newcommand(name, val):
    fmt = "\\newcommand{{\\{name}}}[0]{{{action}}}"
    cmd = fmt.format(name=name, action=val)
    print(cmd)
    return cmd + "\n"

latex_pearsonr = r"r={corr:.2f},\ 95\%\ \mathrm{{CI}}\ [{lower:.2f}, {upper:.2f}]"
latex_beta = r"\beta={beta:.2f}"

cmds = [
    newcommand(
        "ModelVHumanCorr",
        latex_pearsonr.format(**corrs)),
    newcommand(
        "FittedModelVHumanCorr",
        latex_pearsonr.format(**fitted_corrs)),
    newcommand(
        "FittedBeta",
        latex_beta.format(beta=beta))
]

with open(__dest__[1], "w") as fh:
    for cmd in cmds:
        fh.write(cmd)